- Main KITTI readme (old): https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT  
- About Intrinsics: http://ksimek.github.io/2013/08/13/intrinsic/
- Car scheme: http://www.cvlibs.net/publications/Geiger2013IJRR.pdf
- Image with reprojection: https://yadi.sk/i/JAIIsbP5dHAELg
- http://www.cs.toronto.edu/~urtasun/courses/CSC2541/03_odometry.pdf
- https://dsp.stackexchange.com/questions/2736/step-by-step-camera-pose-estimation-for-visual-tracking-and-planar-markers
- https://www.ifi.uzh.ch/dam/jcr:5759a719-55db-4930-8051-4cc534f812b1/VO_Part_I_Scaramuzza.pdf
- https://answers.opencv.org/question/182049/pythonstereo-disparity-quality-problems/
- https://pythonawesome.com/a-toy-implementation-of-a-visual-odometry-vo-pipeline-in-python-2/

## Realizations:
- https://github.com/cgarg92/Stereo-visual-odometry
- https://github.com/ZhenghaoFei/visual_odom
- https://avisingh599.github.io/vision/visual-odometry-full/
- https://github.com/uoip/monoVO-python
- 

## Helpful
- https://www.youtube.com/watch?v=fEaHN8FsW_E

## Paper
- https://www.mdpi.com/1424-8220/18/9/2837/pdf
- http://www.cvlibs.net/projects/autonomous_vision_survey/literature/Cvisic2015ECMR.pdf
- http://cvg.dsi.unifi.it/pdfs/mva2016.pdf
- https://www.scitepress.org/Papers/2018/66236/66236.pdf
- https://pdfs.semanticscholar.org/879f/70a13aa7e461ec2425093f47475ac601a550.pdf

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints

%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib as mpl
print(mpl.style.available)
mpl.style.use('grayscale')


from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR, sequence_id="00")

In [ ]:
c_frame_idx = 0
n_frame_idx = c_frame_idx+1
c_l_img, c_r_img = dataset.get_images(c_frame_idx)
n_l_img, n_r_img = dataset.get_images(n_frame_idx)
PL, PR = dataset.get_P_matrix()
Q_left = dataset.get_left_Q_matrix()
c_pose = dataset.get_poses()[c_frame_idx]
n_pose = dataset.get_poses()[n_frame_idx]
C_left, _ = dataset.get_С_matrix()
_, gt_trnsl = dataset._get_transform(c_frame_idx, n_frame_idx)

vo = VisualOdometry()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(c_l_img)
ax2.imshow(n_l_img)

In [ ]:
c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()
draw_keypoints(c_img_canvas, n_img_canvas, c_feats, n_feats)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
cl_feats, cr_feats = vo.get_disparity_features(c_l_img, c_r_img)

cl_img_canvas = c_l_img.copy()
cr_img_canvas = c_r_img.copy()
draw_keypoints(cl_img_canvas, cr_img_canvas, cl_feats, cr_feats)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(cl_img_canvas)
ax2.imshow(cr_img_canvas)

c_pnts_3d = vo.get_3d_points(cl_feats, cr_feats, PL, PR)
print(c_pnts_3d)

In [ ]:
cl_feats, cr_feats, nl_feats, nr_feats = vo.get_circular_features(c_l_img, c_r_img, n_l_img, n_r_img)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()
draw_keypoints(c_img_canvas, n_img_canvas, cl_feats, nl_feats)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)


# c_feats = np.vstack((cl_feats, cr_feats))
# n_feats = np.vstack((nl_feats, nr_feats))
# flows = np.linalg.norm(cl_feats-nl_feats, axis=1)
# print(flows[flows > 20].size/flows.size)
# print(flows)
# idxs = flows > 35

# cl_feats = cl_feats[idxs]
# nl_feats = nl_feats[idxs]
# nr_feats = nr_feats[idxs]

n_pnts_3d = vo.get_3d_points(nl_feats, nr_feats, PL, PR)
c_pnts_2d = cl_feats.copy()


solver_data = {}
transform = vo.get_transform(c_pnts_2d, None, None, n_pnts_3d, C_left, type_='PnPRansac', solver_data=solver_data)
print(f'gt_t: {gt_trnsl}')
print(f'pred_t: {transform[:3,3]}')
n_pred_pose = vo.get_next_pose(transform, c_pose)
err = np.abs(n_pose-n_pred_pose)
print(err[:3,:3])
print(err[:3,3])

inl_idxs = solver_data['inliers'][:,0]
cl_feats_rnsc = cl_feats[inl_idxs]
cr_feats_rnsc = cr_feats[inl_idxs]
nl_feats_rnsc = nl_feats[inl_idxs]
nr_feats_rnsc = nr_feats[inl_idxs]

fix_threshold = 35
flows = np.linalg.norm(cl_feats_rnsc-nl_feats_rnsc, axis=1)
print(flows[flows > fix_threshold].size/flows.size)

show_3d_features(c_pnts_3d, n_pnts_3d, C_left)
plt.figure(figsize=[14,5])
show_flow(cl_feats_rnsc, nl_feats_rnsc, n_l_img)

In [ ]:
l_img = c_l_img.copy()
r_img = c_r_img.copy()


orb = cv2.ORB_create()
matcher= cv2.BFMatcher()

# FLANN_INDEX_KDTREE = 1
# index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# search_params = dict(checks=50)
# matcher = cv2.FlannBasedMatcher(index_params, search_params)

if len(l_img.shape) == 3 and l_img.shape[2] > 1:
    l_img = cv2.cvtColor(l_img, cv2.COLOR_RGB2GRAY)
    r_img = cv2.cvtColor(r_img, cv2.COLOR_RGB2GRAY)

l_kp, l_des = orb.detectAndCompute(l_img, None)
r_kp, r_des = orb.detectAndCompute(r_img, None)

l_des = np.float32(l_des)
r_des = np.float32(r_des)


matches = matcher.knnMatch(l_des, r_des, k=2)
# matches = sorted(matches, key = lambda x:x.distance)
l_pts = []
r_pts = []

good_matches = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

l_pts = np.array([f.pt for f in l_kp], dtype=int)
r_pts = np.array([f.pt for f in r_kp], dtype=int)

matches_img = draw_matches(l_img, l_kp, r_img, r_kp, good_matches, color=0, radius=5)
plt.figure(figsize=[14,8])
plt.imshow(matches_img)

In [ ]:
c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(c_depth_frame[:,:,2])
ax2.imshow(n_depth_frame[:,:,2])

In [ ]:
def show_3d_features(c_pnts3d, n_pnts3d, C_left):
    P_left = np.hstack((C_left, np.transpose([[0, 0, 0]])))

    c_pnts_2d = vo.reproject_3d_to_2d(c_pnts3d, P_left)
    n_pnts_2d = vo.reproject_3d_to_2d(n_pnts3d, P_left)

    c_img_canvas = c_l_img.copy()
    n_img_canvas = n_l_img.copy()
    # Check after clique and projection
    draw_keypoints(c_img_canvas, n_img_canvas, c_pnts_2d, n_pnts_2d)
    # # Rendering valid features
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
    ax1.imshow(c_img_canvas)
    ax2.imshow(n_img_canvas)
    
def show_flow(c_pnts2d, n_pnts2d, img):
    img = img.copy()
    
    r = 3
    thickness = 2
    c = (0, 255, 0)        
    for i in range(len(c_pnts2d)):
        c_pnt = c_pnts2d[i]
        n_pnt = n_pnts2d[i]
        end1 = tuple(np.round(c_pnt).astype(int))
        end2 = tuple(np.round(n_pnt).astype(int))
        cv2.line(img, end1, end2, c, thickness)
        cv2.circle(img, end1, r, c, thickness)
        cv2.circle(img, end2, r, c, thickness)
    
    plt.imshow(img)

In [ ]:
### Get 3D points 
c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

ft_idxs = c_ft_idxs & n_ft_idxs

c_pnts_3d = c_pnts_3d[ft_idxs]
n_pnts_3d = n_pnts_3d[ft_idxs]
c_feats = c_feats[ft_idxs]
n_feats = n_feats[ft_idxs]

# Find essential to get first estimation
T, mask = vo.get_relative_transform(n_feats, c_feats, C_left)
print(T)

# rnsc_ess_idxs = mask[:,0]
# c_feats_rnsc_ess = c_feats[rnsc_ess_idxs]
# n_feats_rnsc_ess = n_feats[rnsc_ess_idxs]
# c_pnts_3d_ess = c_pnts_3d[rnsc_ess_idxs]
# n_pnts_3d_ess = n_pnts_3d[rnsc_ess_idxs]

solver_data = {
#     'tvec': t
}
transform = vo.get_transform(c_feats, n_feats, c_pnts_3d, n_pnts_3d, C_left, type_='PnPRansac', solver_data=solver_data)
print(f'gt_t: {gt_trnsl}')
print(f'pred_t: {transform[:3,3]}')
n_pred_pose = vo.get_next_pose(transform, c_pose)
err = np.abs(n_pose-n_pred_pose)
print(err[:3,:3])
print(err[:3,3])

inl_idxs = solver_data['inliers'][:,0]
c_feats_rnsc = c_feats[inl_idxs]
n_feats_rnsc = n_feats[inl_idxs]

show_3d_features(c_pnts_3d, n_pnts_3d, C_left)
plt.figure(figsize=[14,5])
show_flow(c_feats_rnsc, n_feats_rnsc, n_l_img)